# **Data Cleaning Script**



In [2]:
# Loading the libraries
import pandas as pd
import numpy as np

In [4]:
# Setting pandas options to display all columns for better inspection
pd.set_option('display.max_columns', None)

In [6]:
# Loading Data
df_raw = pd.read_csv('delhivery_data.csv')

/tmp/ipython-input-158388052.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv('delhivery_data.csv')


In [22]:
df_raw.shape

(114711, 24)

In [ ]:
# Initial Inspection
# Data Types and Null Values
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32776 entries, 0 to 32775
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   data                            32776 non-null  object 
 1   trip_creation_time              32776 non-null  object 
 2   route_schedule_uuid             32776 non-null  object 
 3   route_type                      32776 non-null  object 
 4   trip_uuid                       32776 non-null  object 
 5   source_center                   32776 non-null  object 
 6   source_name                     32678 non-null  object 
 7   destination_center              32776 non-null  object 
 8   destination_name                32714 non-null  object 
 9   od_start_time                   32776 non-null  object 
 10  od_end_time                     32776 non-null  object 
 11  start_scan_to_end_scan          32776 non-null  float64
 12  is_cutoff                       

### Issues Found: (by inspecting info)
*   'source_name' & 'destination_name' have missing values.
*   Time-related columns (trip_creation_time, od_start_time, od_end_time) are 'object' (string) type, not datetime, it will prevents calculations
*   Soo many 'unknown' columns exist ('is_cutoff', 'factor',etc)
*   Data dictionary confirms data is at a 'segment' level. The 'actual_time' and 'osrm_time' columns are cumulative and are confusing
* The 'segment_actual_time' and 'segment_osrm_time' are clearer and better for doing analysis.




In [9]:
# The following are details of the columns we have in dataset
"""
DATA DICTIONARY

• data: Tells whether the data is testing or training data
• trip_creation_time: Timestamp of trip creation
• route_schedule_uuid: Unique Id for a particular route schedule
• route_type: Transportation type
    • FTL – Full Truck Load: FTL shipments get to the destination sooner,
      as the truck is making no other pickups or drop-offs along the way.
    • Carting: Handling system consisting of small vehicles (carts)
• trip_uuid: Unique ID given to a particular trip
• source_center: Source ID of trip origin
• source_name: Source Name of trip origin
• destination_cente: Destination ID
• destination_name: Destination Name
• od_start_time: Trip start time
• od_end_time: Trip end time
• start_scan_to_end_scan: Time taken to deliver from source to destination (Cumulative)
• actual_distance_to_destination: Distance in Kms (Cumulative)
• actual_time: Actual time taken to complete the delivery (Cumulative)
• osrm_time: An open-source routing engine time calculator (Cumulative)
• osrm_distance: An open-source routing engine distance calculator (Cumulative)
• segment_actual_time: Time taken for the subset of the package delivery
• segment_osrm_time: The OSRM segment time
• segment_osrm_distance: The OSRM segment distance

 we have to drop thesee Unknown/Ignored Columns
• is_cutoff, cutoff_factor, cutoff_timestamp, factor, segment_factor

"""

'\nDATA DICTIONARY\n\n• data: Tells whether the data is testing or training data\n• trip_creation_time: Timestamp of trip creation\n• route_schedule_uuid: Unique Id for a particular route schedule\n• route_type: Transportation type\n    • FTL – Full Truck Load: FTL shipments get to the destination sooner,\n      as the truck is making no other pickups or drop-offs along the way.\n    • Carting: Handling system consisting of small vehicles (carts)\n• trip_uuid: Unique ID given to a particular trip\n• source_center: Source ID of trip origin\n• source_name: Source Name of trip origin\n• destination_cente: Destination ID\n• destination_name: Destination Name\n• od_start_time: Trip start time\n• od_end_time: Trip end time\n• start_scan_to_end_scan: Time taken to deliver from source to destination (Cumulative)\n• actual_distance_to_destination: Distance in Kms (Cumulative)\n• actual_time: Actual time taken to complete the delivery (Cumulative)\n• osrm_time: An open-source routing engine time

### Checking for  Missing Values and Duplicates


In [10]:
# Check missing values
print(df_raw.isnull().sum())

data                                0
trip_creation_time                  0
route_schedule_uuid                 0
route_type                          1
trip_uuid                           1
source_center                       1
source_name                       213
destination_center                  1
destination_name                  167
od_start_time                       1
od_end_time                         1
start_scan_to_end_scan              1
is_cutoff                           1
cutoff_factor                       1
cutoff_timestamp                    1
actual_distance_to_destination      1
actual_time                         1
osrm_time                           1
osrm_distance                       1
factor                              1
segment_actual_time                 1
segment_osrm_time                   1
segment_osrm_distance               1
segment_factor                      1
dtype: int64


In [12]:
# Checking duplicates
df_raw.duplicated().sum()

np.int64(0)

## Cleaning and Transformation


In [36]:
df_clean = df_raw.copy()

### Handling Missing Values


In [37]:
df_clean.dropna(subset=['source_name', 'destination_name'], inplace=True)

In [23]:
df_clean.shape

(114335, 24)

### Drop unknown columns





In [47]:
columns_to_drop = ['is_cutoff','cutoff_factor','cutoff_timestamp',
                   'factor','segment_factor']

print("Columns in df_clean before dropping:", df_clean.columns.tolist())

# df_clean = df_clean.drop(columns=columns_to_drop)
# print("Dropped irrelevant/unknown columns.")

Columns in df_clean before dropping: ['data', 'trip_creation_time', 'route_schedule_uuid', 'route_type', 'trip_uuid', 'source_center', 'source_name', 'destination_center', 'destination_name', 'od_start_time', 'od_end_time', 'start_scan_to_end_scan', 'actual_distance_to_destination', 'actual_time', 'osrm_time', 'osrm_distance', 'segment_actual_time', 'segment_osrm_time', 'segment_osrm_distance', 'actual_trip_duration_hours', 'actual_time_hours', 'osrm_time_hours', 'delay_vs_expected_hours', 'source_state', 'destination_state', 'route']


### Check columns before dropping

### Convert data type(Time)

In [39]:
''' To perform time based analysis,
 we must convert time column from string to datetime object '''

df_clean['trip_creation_time'] = pd.to_datetime(df_clean['trip_creation_time'])
df_clean['od_start_time'] = pd.to_datetime(df_clean['od_start_time'])
df_clean['od_end_time'] = pd.to_datetime(df_clean['od_end_time'])
print("Converted time columns to datetime objects.")

Converted time columns to datetime objects.


### Handle Duplicates

In [40]:
# df_clean = df_clean.drop_duplicates()
''' Earler we found there were 0 duplicates, so when we have duplicates,
we will use this '''

' Earler we found there were 0 duplicates, so when we have duplicates,\nwe will use this '

## Feature Engineering(Prep for Analysis)

In [41]:
# Converting seconds to hours(by calc diff b/w seconds)- for readability
df_clean['actual_trip_duration_hours'] = (df_clean['od_end_time'] -
                                          df_clean['od_start_time']).dt.total_seconds() / 3600

In [42]:
# Calculating delay vs expected hours
# we compare actual_time(provided) and orsm_time(expected)
# A +ve number means a delay
# we have to convert minutes to hours coz they are in minutes

df_clean['actual_time_hours'] = df_clean['actual_time'] / 60
df_clean['osrm_time_hours'] = df_clean['osrm_time'] / 60

df_clean['delay_vs_expected_hours'] = (
    df_clean['actual_time_hours'] - df_clean['osrm_time_hours']
)

In [48]:
'''Extracting Route Information
The 'source_name' and 'destination_name' have state info we can extract.
for eg, look into this'Anand_VUNagar_DC (Gujarat)'
'''

df_clean['source_state'] = df_clean['source_name'].apply(
    lambda x: x.split('(')[-1].replace(')', ''))

df_clean['destination_state'] = df_clean['destination_name'].apply(
    lambda x: x.split('(')[-1].replace(')', ''))

# Creating a 'route' column for easy grouping
df_clean['route'] = (
    df_clean['source_state'] + ' -> ' + df_clean['destination_state'])

print("Created new features: 'actual_trip_duration_hours', 'delay_vs_expected_hours', 'route'.")

Created new features: 'actual_trip_duration_hours', 'delay_vs_expected_hours', 'route'.


###Final Validation

In [51]:
df_clean.info

<bound method DataFrame.info of             data         trip_creation_time  \
0       training 2018-09-20 02:35:36.476840   
1       training 2018-09-20 02:35:36.476840   
2       training 2018-09-20 02:35:36.476840   
3       training 2018-09-20 02:35:36.476840   
4       training 2018-09-20 02:35:36.476840   
...          ...                        ...   
114705      test 2018-09-27 03:50:13.599818   
114706      test 2018-09-27 03:50:13.599818   
114707      test 2018-09-27 03:50:13.599818   
114708      test 2018-09-27 03:50:13.599818   
114709      test 2018-09-27 03:50:13.599818   

                                      route_schedule_uuid route_type  \
0       thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...    Carting   
1       thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...    Carting   
2       thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...    Carting   
3       thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...    Carting   
4       thanos::sroute:eb7bfc78-b351-4c0e-a951-fa3d5c3...    Carting   
...                                                   ...        ...   
114705  thanos::sroute:cce26bb2-2365-4c9c-88f4-74322d1...    Carting   
114706  thanos::sroute:cce26bb2-2365-4c9c-88f4-74322d1...    Carting   
114707  thanos::sroute:cce26bb2-2365-4c9c-88f4-74322d1...    Carting   
114708  thanos::sroute:cce26bb2-2365-4c9c-88f4-74322d1...    Carting   
114709  thanos::sroute:cce26bb2-2365-4c9c-88f4-74322d1...    Carting   

                      trip_uuid source_center  \
0       trip-153741093647649320  IND388121AAA   
1       trip-153741093647649320  IND388121AAA   
2       trip-153741093647649320  IND388121AAA   
3       trip-153741093647649320  IND388121AAA   
4       trip-153741093647649320  IND388121AAA   
...                         ...           ...   
114705  trip-153802021359954030  IND516115AAA   
114706  trip-153802021359954030  IND516115AAA   
114707  trip-153802021359954030  IND516115AAA   
114708  trip-153802021359954030  IND516115AAA   
114709  trip-153802021359954030  IND516115AAA   

                               source_name destination_center  \
0               Anand_VUNagar_DC (Gujarat)       IND388620AAB   
1               Anand_VUNagar_DC (Gujarat)       IND388620AAB   
2               Anand_VUNagar_DC (Gujarat)       IND388620AAB   
3               Anand_VUNagar_DC (Gujarat)       IND388620AAB   
4               Anand_VUNagar_DC (Gujarat)       IND388620AAB   
...                                    ...                ...   
114705  Rajampet_RSRoad_D (Andhra Pradesh)       IND517501AAB   
114706  Rajampet_RSRoad_D (Andhra Pradesh)       IND517501AAB   
114707  Rajampet_RSRoad_D (Andhra Pradesh)       IND517501AAB   
114708  Rajampet_RSRoad_D (Andhra Pradesh)       IND517501AAB   
114709  Rajampet_RSRoad_D (Andhra Pradesh)       IND517501AAB   

                             destination_name              od_start_time  \
0               Khambhat_MotvdDPP_D (Gujarat) 2018-09-20 03:21:32.418600   
1               Khambhat_MotvdDPP_D (Gujarat) 2018-09-20 03:21:32.418600   
2               Khambhat_MotvdDPP_D (Gujarat) 2018-09-20 03:21:32.418600   
3               Khambhat_MotvdDPP_D (Gujarat) 2018-09-20 03:21:32.418600   
4               Khambhat_MotvdDPP_D (Gujarat) 2018-09-20 03:21:32.418600   
...                                       ...                        ...   
114705  Tirupati_Balajicly_I (Andhra Pradesh) 2018-09-27 08:30:35.814948   
114706  Tirupati_Balajicly_I (Andhra Pradesh) 2018-09-27 08:30:35.814948   
114707  Tirupati_Balajicly_I (Andhra Pradesh) 2018-09-27 08:30:35.814948   
114708  Tirupati_Balajicly_I (Andhra Pradesh) 2018-09-27 08:30:35.814948   
114709  Tirupati_Balajicly_I (Andhra Pradesh) 2018-09-27 08:30:35.814948   

                      od_end_time  start_scan_to_end_scan  \
0      2018-09-20 04:47:45.236797                    86.0   
1      2018-09-20 04:47:45.236797                    86.0   
2      2018-09-20 04:47:45.236797                    86.0   
3      2018

### Saving the cleansed data to new file for analysis phase

In [54]:
df_clean.to_csv('delhivery_data_CLEANED.csv', index=False)
print("\nSuccessfully cleaned data and saved to 'delhivery_data_CLEANED.csv'")


Successfully cleaned data and saved to 'delhivery_data_CLEANED.csv'
